In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5 MB 5.6 kB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 3.8 MB 63.7 MB/s 
     |████████████████████████████████| 503 kB 65.7 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=b186ef2c47419d4252691b996390d0ad19e37024210c233470058f4ae933756f
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succe

In [ ]:
!wget https://mycityreport.s3-ap-northeast-1.amazonaws.com/02_RoadDamageDataset/public_data/IEEE_bigdata_RDD2020/train.tar.gz

--2022-03-29 00:19:27--  https://mycityreport.s3-ap-northeast-1.amazonaws.com/02_RoadDamageDataset/public_data/IEEE_bigdata_RDD2020/train.tar.gz
Resolving mycityreport.s3-ap-northeast-1.amazonaws.com (mycityreport.s3-ap-northeast-1.amazonaws.com)... 52.219.17.46
Connecting to mycityreport.s3-ap-northeast-1.amazonaws.com (mycityreport.s3-ap-northeast-1.amazonaws.com)|52.219.17.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1472626254 (1.4G) [application/x-tar]
Saving to: ‘train.tar.gz’

train.tar.gz         92%[=================>  ]   1.27G  8.19MB/s    eta 9s     

In [ ]:

!tar -xzvf  'train.tar.gz'

In [ ]:
import tensorflow as tf
import tensorflow.contrib as tf_contrib
from tensorflow.contrib.layers import variance_scaling_initializer as he_init


##################################################################################
# Layer
##################################################################################

def linear(x, units, use_bias=True, activation_fn='leaky', is_training=True, norm_fn='instance', scope='linear'):
    with tf.variable_scope(scope):
        x = tf.layers.flatten(x)
        x = tf.layers.dense(x, units=units, kernel_initializer=tf_contrib.layers.variance_scaling_initializer(), kernel_regularizer=tf_contrib.layers.l2_regularizer(scale=0.0001), use_bias=use_bias)

        if norm_fn == 'instance' :
            x = instance_norm(x, 'instance')
        if norm_fn == 'batch' :
            x = batch_norm(x, is_training, 'batch')
        
        if activation_fn!='None':
            x = activation(x, activation_fn)
        return x
                
def conv(x, channels, kernel=3, stride=2, pad=0, activation_fn='leaky', is_training=True, norm_fn='instance', is_spectral_norm=False, use_bias=True, scope='conv_0') :
    with tf.variable_scope(scope):
        if scope.__contains__("discriminator") :
            weight_init = tf.random_normal_initializer(mean=0.0, stddev=0.02)
        else :
            weight_init = tf_contrib.layers.xavier_initializer()

        x = tf.pad(x, [[0,0], [pad, pad], [pad, pad], [0,0]])
        
        if is_spectral_norm == True :
            w = tf.get_variable("kernel", shape=[kernel, kernel, x.get_shape()[-1], channels], initializer=weight_init,
                                regularizer=None)
            bias = tf.get_variable("bias", [channels], initializer=tf.constant_initializer(0.0))
            x = tf.nn.conv2d(input=x, filter=spectral_norm(w),
                             strides=[1, stride, stride, 1], padding='VALID')
            if use_bias:
                x = tf.nn.bias_add(x, bias)

        else :
            x = tf.layers.conv2d(inputs=x, filters=channels,
                                 kernel_size=kernel, kernel_initializer=weight_init,
                                 kernel_regularizer=tf_contrib.layers.l2_regularizer(scale=0.0001),
                                 strides=stride, use_bias=use_bias)

        if norm_fn == 'instance' :
            x = instance_norm(x, 'instance')
        if norm_fn == 'batch' :
            x = batch_norm(x, is_training, 'batch')
        if norm_fn == 'layer' :
            x = layer_norm(x, scope='layer_norm') 

        if activation_fn!='None':
            x = activation(x, activation_fn)
            
        return x

def deconv(x, channels, kernel=4, stride=2, activation_fn='leaky', is_training=True, norm_fn='instance', use_bias=True, scope='deconv_0'):
    with tf.variable_scope(scope):
        weight_init = tf.random_normal_initializer(mean=0.0, stddev=0.02)
        x = tf.layers.conv2d_transpose(inputs=x, filters=channels,
                                       kernel_size=kernel, kernel_initializer=weight_init, 
                                       kernel_regularizer=tf_contrib.layers.l2_regularizer(scale=0.0001),
                                       strides=stride, padding='SAME', use_bias=use_bias)

        if norm_fn == 'instance' :
            x = instance_norm(x, 'instance')
        if norm_fn == 'batch' :
            x = batch_norm(x, is_training, 'batch')
        if norm_fn == 'layer' :
            x = layer_norm(x, scope='layer_norm') 

        if activation_fn!='None':
            x = activation(x, activation_fn)

        return x
    
def gaussian_noise_layer(mu, scope='noise'):
    with tf.variable_scope(scope):
        sigma = 0.4#tf.get_variable("sigma", [1], tf.float32, tf.random_normal_initializer(stddev=0.02))
        gaussian_random_vector = tf.random_normal(shape=tf.shape(mu), mean=0.0, stddev=1.0, dtype=tf.float32)
        return mu + sigma * gaussian_random_vector

##################################################################################
# Residual-block
##################################################################################

def resblock(x_init, channels, kernel=3, stride=1, pad=1, dropout_ratio=0.0, is_training=True, norm_fn='instance', is_spectral_norm=False, use_bias=True, scope='resblock') :
    assert norm_fn in ['instance', 'batch', 'weight', 'spectral', 'None', 'layer']
    with tf.variable_scope(scope) :
        with tf.variable_scope('res1') :
            x = conv(x_init, channels, kernel=3, stride=1, pad=1, activation_fn='relu', norm_fn=norm_fn, is_spectral_norm=is_spectral_norm, use_bias=use_bias) 
            
        with tf.variable_scope('res2') :
            x = conv(x, channels, kernel=3, stride=1, pad=1, activation_fn='None', norm_fn=norm_fn, is_spectral_norm=is_spectral_norm, use_bias=use_bias) 

        if dropout_ratio > 0.0 :
            x = tf.layers.dropout(x, rate=dropout_ratio, training=is_training)

        return x + x_init

def adaptive_resblock(x_init, channels, mu, sigma, is_spectral_norm=False, use_bias=True, scope='adaptive_resblock') :
    with tf.variable_scope(scope):
        with tf.variable_scope('res1'):
            x = conv(x_init, channels, kernel=3, stride=1, pad=1, activation_fn='None', norm_fn='None', is_spectral_norm=is_spectral_norm, use_bias=use_bias) 
            x = adaptive_instance_norm(x, mu, sigma)
            x = relu(x)

        with tf.variable_scope('res2'):
            x = conv(x, channels, kernel=3, stride=1, pad=1, activation_fn='None', norm_fn='None', is_spectral_norm=is_spectral_norm, use_bias=use_bias) 
            x = adaptive_instance_norm(x, mu, sigma)

        return x + x_init
    
def hw_flatten(x):
    return tf.reshape(x, shape=[x.shape[0], -1, x.shape[-1]])

##################################################################################
# Activation function
##################################################################################
        
def activation(x, activation_fn='leaky') :
    assert activation_fn in ['relu', 'leaky', 'tanh', 'sigmoid', 'swish', 'selu', 'None']
    if activation_fn == 'leaky':
        x = lrelu(x)

    if activation_fn == 'relu':
        x = relu(x)

    if activation_fn == 'sigmoid':
        x = sigmoid(x)

    if activation_fn == 'tanh' :
        x = tanh(x)

    if activation_fn == 'swish' :
        x = swish(x)

    if activation_fn == 'selu' :
        x = selu(x)

    return x

def lrelu(x, alpha=0.01) :
    # pytorch alpha is 0.01
    return tf.nn.leaky_relu(x, alpha)

def relu(x) :
    return tf.nn.relu(x)

def sigmoid(x) :
    return tf.sigmoid(x)

def tanh(x) :
    return tf.tanh(x)

def swish(x) :
    return x * sigmoid(x)

def selu(x) :
    return tf.nn.selu(x)

##################################################################################
# Normalization function
##################################################################################

def batch_norm(x, is_training=False, scope='batch_nom') :
    return tf_contrib.layers.batch_norm(x,
                                        decay=0.9, epsilon=1e-05,
                                        center=True, scale=True, updates_collections=None,
                                        is_training=is_training, scope=scope)

def instance_norm(x, scope='instance_nom') :
    return tf_contrib.layers.instance_norm(x,
                                           epsilon=1e-05,
                                           center=True, scale=True,
                                           scope=scope)

def layer_norm(x, scope='layer_norm') :
    return tf_contrib.layers.layer_norm(x,
                                        center=True, scale=True,
                                        scope=scope)

def adaptive_instance_norm(content, gamma, beta, epsilon=1e-5):
    # gamma, beta = style_mean, style_std from MLP

    c_mean, c_var = tf.nn.moments(content, axes=[1, 2], keep_dims=True)
    c_std = tf.sqrt(c_var + epsilon)

    return gamma * ((content - c_mean) / c_std) + beta

def l2_norm(v, eps=1e-12):
    return v / (tf.reduce_sum(v ** 2) ** 0.5 + eps)

def spectral_norm(w, iteration=1):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])

    u = tf.get_variable("u", [1, w_shape[-1]], initializer=tf.truncated_normal_initializer(), trainable=False)

    u_hat = u
    v_hat = None
    for i in range(iteration):
        """
        power iteration
        Usually iteration = 1 will be enough
        """
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = l2_norm(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = l2_norm(u_)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))
    w_norm = w / sigma

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = tf.reshape(w_norm, w_shape)

    return w_norm

##################################################################################
# Sampling
##################################################################################

def down_sample(x) :
    return tf.layers.average_pooling2d(x, pool_size=3, strides=2, padding='SAME')

def up_sample(x, scale_factor=2, metod='neighbor'):
    _, h, w, _ = x.get_shape().as_list()
    new_size = [h * scale_factor, w * scale_factor]
    if metod=='bilinear':
        return tf.image.resize_bilinear(x, size=new_size)
    else:
        return tf.image.resize_nearest_neighbor(x, size=new_size)

def adaptive_avg_pooling(x):
    # global average pooling
    gap = tf.reduce_mean(x, axis=[1, 2], keep_dims=True)

    return gap

def KL_divergence(x) :
#    mu = tf.reduce_mean(x)
#    sigma_2 = tf.reduce_mean(tf.square(x-mu))
#    KL_divergence = 0.5 * (tf.square(mu) + sigma_2 - tf.log(sigma_2) - 1)
#    loss = tf.reduce_mean(KL_divergence)

    loss = tf.square(tf.reduce_mean(x))

    return loss

##################################################################################
# Loss function
##################################################################################

def regularization_loss(scope_name) :
    """
    If you want to use "Regularization"
    g_loss += regularization_loss('generator')
    d_loss += regularization_loss('discriminator')
    """
    collection_regularization = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)

    loss = []
    for item in collection_regularization :
        if scope_name in item.name :
            loss.append(item)

    return tf.reduce_sum(loss)

def L1_loss(x, y) :
    loss = tf.reduce_mean(tf.abs(x - y))
    return loss


def discriminator_loss(loss_func, real, fake):
    real_loss = 0
    fake_loss = 0

    if loss_func.__contains__('wgan') :
        real_loss = -tf.reduce_mean(real)
        fake_loss = tf.reduce_mean(fake)

    if loss_func == 'lsgan' :
        real_loss = tf.reduce_mean(tf.squared_difference(real, 1.0))
        fake_loss = tf.reduce_mean(tf.square(fake))

    if loss_func == 'gan' or loss_func == 'dragan' :
        real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(real), logits=real))
        fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(fake), logits=fake))

    if loss_func == 'hinge' :
        real_loss = tf.reduce_mean(relu(1.0 - real))
        fake_loss = tf.reduce_mean(relu(1.0 + fake))

    loss = real_loss + fake_loss

    return loss

def generator_loss(loss_func, fake):
    fake_loss = 0

    if loss_func.__contains__('wgan') :
        fake_loss = -tf.reduce_mean(fake)

    if loss_func == 'lsgan' :
        fake_loss = tf.reduce_mean(tf.squared_difference(fake, 1.0))

    if loss_func == 'gan' or loss_func == 'dragan' :
        fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(fake), logits=fake))

    if loss_func == 'hinge' :
        fake_loss = -tf.reduce_mean(fake)

    loss = fake_loss

    return loss

def classification_loss1(logit, label) :
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=label, logits=logit))
    return loss

def classification_loss2(logit, label) :
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=label, logits=logit))
    return loss

# def classification_loss2(label_o, label_p) :
#     loss = tf.reduce_mean(-label_o*tf.log(tf.clip_by_value(label_p,1e-10,1.0))-(1-label_o)*tf.log(tf.clip_by_value((1-label_p),1e-10,1.0)))
#     return loss


In [ ]:
!pip install np_utils

In [ ]:
import tensorflow as tf
from tensorflow.contrib import slim
from scipy import misc
import os, random
import numpy as np
from glob import glob
import np_utils as np_utils
try:
    import xml.etree.cElementTree as ET  #解析xml的c语言版的模块
except ImportError:
    import xml.etree.ElementTree as ET

class ImageData:

    def __init__(self, data_path, img_shape=(64,64,1), augment_flag=False, data_type='None', img_type='jpg', pad_flag=False, label_size=8):
        self.data_path = data_path
        self.data_type = data_type
        self.img_shape = img_shape
        self.img_h = img_shape[0]
        self.img_w = img_shape[1]
        self.channels = img_shape[2]
        self.augment_flag = augment_flag
        self.img_type = img_type
        self.pad_flag = pad_flag
        self.label_size = label_size
        self.class_names = os.listdir(self.data_path)
        self.train_dataset = []
        self.train_label = []
        images = []
        for cl_name in self.class_names:
            img_names = os.listdir(os.path.join(self.data_path, cl_name))
            for img_name in img_names:
                self.train_dataset.append(os.path.abspath(os.path.join(self.data_path, cl_name, img_name)))
                hot_cl_name = self.get_class_one_hot(cl_name)
                self.train_label.append(hot_cl_name)
        self.train_label = np.reshape(self.train_label, (len(self.train_label), self.label_size))

    def get_class_one_hot(self, class_str):
        label_encoded = self.class_names.index(class_str)
        label_hot = np_utils.to_categorical(label_encoded, len(self.class_names))
        label_hot = label_hot

        return label_hot

    def image_processing(self, filename, label):
        x = tf.read_file(filename)
        if self.img_type == 'jpg':
            x_decode = tf.image.decode_jpeg(x, channels=self.channels)
        if self.img_type == 'png':
            x_decode = tf.image.decode_png(x, channels=self.channels)
        if self.img_type == 'bmp':
            x_decode = tf.image.decode_bmp(x)
            if self.channels == 1 :
                x_decode = tf.image.rgb_to_grayscale(x_decode)
        img = tf.image.resize_images(x_decode, [self.img_h, self.img_w])
        img = tf.reshape(img, [self.img_h, self.img_w, self.channels])
        img = tf.cast(img, tf.float32) / 127.5 - 1

        if self.augment_flag :
            img = tf.cond(pred=tf.greater_equal(tf.random_uniform(shape=[], minval=0.0, maxval=1.0), 0.5),
                          true_fn=lambda: augmentation(img),
                          false_fn=lambda: img)
        
        return img, label
    
def one_hot(batch_size, mask_size, location):
    l = tf.constant([location])
    m = tf.one_hot(l,mask_size,1.,0.)
    m = tf.tile(m,[batch_size,1])
    return m
    
def load_test_data(image_path, size_h=256, size_w=256):
    img = misc.imread(image_path, mode='RGB')
    img = misc.imresize(img, [size_h, size_w])
    img = np.expand_dims(img, axis=0)
    img = preprocessing(img)

    return img

def preprocessing(x):
    x = x/127.5 - 1 # -1 ~ 1
    return x

def augmentation(image):
    seed = random.randint(0, 2 ** 31 - 1)
    image = tf.image.random_flip_left_right(image, seed=seed)
#    image = tf.image.random_brightness(image,max_delta=0.2)
#    image = tf.image.random_contrast(image, 0.5, 1.5)
#    image = tf.clip_by_value(image,-1.,1.)
#    image = tf.image.random_saturation(image, 0, 0.3)
    return image
											         
def GetAnnotBoxLoc(AnotPath):#AnotPath VOC标注文件路径
    tree = ET.ElementTree(file=AnotPath)  #打开文件，解析成一棵树型结构
    root = tree.getroot()#获取树型结构的根
    ObjectSet=root.findall('object')#找到文件中所有含有object关键字的地方，这些地方含有标注目标
    ObjBndBoxSet={} #以目标类别为关键字，目标框为值组成的字典结构
    for Object in ObjectSet:
        ObjName=Object.find('name').text
        BndBox=Object.find('bndbox')
        x1 = int(BndBox.find('xmin').text)#-1 #-1是因为程序是按0作为起始位置的
        y1 = int(BndBox.find('ymin').text)#-1
        x2 = int(BndBox.find('xmax').text)#-1
        y2 = int(BndBox.find('ymax').text)#-1
        BndBoxLoc=[x1,y1,x2,y2]
        if ObjName in ObjBndBoxSet:
            ObjBndBoxSet[ObjName].append(BndBoxLoc)#如果字典结构中含有这个类别了，那么这个目标框要追加到其值的末尾
        else:
            ObjBndBoxSet[ObjName]=[BndBoxLoc]#如果字典结构中没有这个类别，那么这个目标框就直接赋值给其值吧
    return ObjBndBoxSet

def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def inverse_transform(images):
    return (images+1.) / 2

def imsave(images, size, path):
    return misc.imsave(path, merge(images, size))

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[h*j:h*(j+1), w*i:w*(i+1), :] = image

    return img

def show_all_variables():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir

def str2bool(x):
    return x.lower() in ('true')

def summary(tensor_collection, summary_type=['mean', 'stddev', 'max', 'min', 'sparsity', 'histogram']):
    """
    usage:

    1. summary(tensor)

    2. summary([tensor_a, tensor_b])

    3. summary({tensor_a: 'a', tensor_b: 'b})
    """

    def _summary(tensor, name, summary_type=['mean', 'stddev', 'max', 'min', 'sparsity', 'histogram']):
        """ Attach a lot of summaries to a Tensor. """

        if name is None:
            # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
            # session. This helps the clarity of presentation on tensorboard.
            name = re.sub('%s_[0-9]*/' % 'tower', '', tensor.name)
            name = re.sub(':', '-', name)

        with tf.name_scope('summary_' + name):
            summaries = []
            if len(tensor.shape) == 0:
                summaries.append(tf.summary.scalar(name, tensor))
            else:
                if 'mean' in summary_type:
                    mean = tf.reduce_mean(tensor)
                    summaries.append(tf.summary.scalar(name + '/mean', mean))
                if 'stddev' in summary_type:
                    mean = tf.reduce_mean(tensor)
                    stddev = tf.sqrt(tf.reduce_mean(tf.square(tensor - mean)))
                    summaries.append(tf.summary.scalar(name + '/stddev', stddev))
                if 'max' in summary_type:
                    summaries.append(tf.summary.scalar(name + '/max', tf.reduce_max(tensor)))
                if 'min' in summary_type:
                    summaries.append(tf.summary.scalar(name + '/min', tf.reduce_min(tensor)))
                if 'sparsity' in summary_type:
                    summaries.append(tf.summary.scalar(name + '/sparsity', tf.nn.zero_fraction(tensor)))
                if 'histogram' in summary_type:
                    summaries.append(tf.summary.histogram(name, tensor))
            return tf.summary.merge(summaries)

    if not isinstance(tensor_collection, (list, tuple, dict)):
        tensor_collection = [tensor_collection]
    with tf.name_scope('summaries'):
        summaries = []
        if isinstance(tensor_collection, (list, tuple)):
            for tensor in tensor_collection:
                summaries.append(_summary(tensor, None, summary_type))
        else:
            for tensor, name in tensor_collection.items():
                summaries.append(_summary(tensor, name, summary_type))
        return tf.summary.merge(summaries)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 23 13:57:32 2019
@author: yya
"""

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 11 18:31:42 2019
@author: siat-x
"""

import os
import re
import math
import numpy as np
import tensorflow as tf
from datetime import datetime
import cv2
from glob import glob
from tensorflow.data.experimental import prefetch_to_device, shuffle_and_repeat, map_and_batch
from xml.etree import ElementTree


class GAN:

    def __init__(self, sess, args):
        self.model_name = 'GAN'
        self.sess = sess
        self.gan_type = args.gan_type
        self.epoch = args.epoch
        self.decay_epoch = args.decay_epoch
        self.save_freq = args.save_freq
        self.iteration = args.iteration
        self.init_lr = args.lr
        self.beta1 = 0.5
        
        """ Dir """
        self.checkpoint_dir = args.checkpoint_dir
        self.sample_dir = args.sample_dir
        self.result_dir = args.result_dir
        self.summaries_dir = args.summaries_dir

        """ Weight """
        self.rec_weight = args.rec_weight
        self.cls_weight = args.cls_weight
        self.ld = args.ld
        
        """ Channels """
        self.batch_size = args.batch_size
        self.genn = args.n_gen
        self.disn = args.n_dis
        self.img_size = args.img_size
        self.img_ch = args.img_ch
        self.img_shape = (args.img_size, args.img_size, args.img_ch)
        self.ch = args.ch
            
        """ Dataset """
        self.label_size = args.label_size
        self.augment_flag = args.augment_flag
        self.train_dataset_type = args.dataset_type
        self.train_dataset_img_type = args.dataset_img_type
        self.train_dataset_path = args.dataset_path
           
        """ Test """
        self.test_path = args.test_path
        
##################################################################################
# Generator
##################################################################################
        
    def generator(self, x, c, reuse=False, train=True):
        ys = []
        channel = 256
        norm='batch'
        with tf.variable_scope('generator', reuse=reuse):

            c = tf.cast(tf.reshape(c, shape=[-1, 1, 1, c.shape[-1]]), tf.float32)
            c = tf.tile(c, [1, x.shape[1], x.shape[2], 1])
            x = tf.concat([x, c], axis=-1)
            x = conv(x, channel, kernel=3, stride=1, pad=1, activation_fn='relu', is_training=train, norm_fn=norm, use_bias=False, scope='conv_b')
            
            for i in range(0, self.genn-2) :
                x = up_sample(x, scale_factor=2)
                x = conv(x, channel, kernel=5, stride=1, pad=2, activation_fn='relu', is_training=train, norm_fn=norm, use_bias=False, scope='deconv_'+str(i))

            for i in range(self.genn-2, self.genn) :
                channel = channel/2
                x = up_sample(x, scale_factor=2)
                x = conv(x, channel, kernel=5, stride=1, pad=2, activation_fn='relu', is_training=train, norm_fn=norm, use_bias=False, scope='deconv_'+str(i))
                
            x = conv(x, self.img_ch, kernel=1, stride=1, pad=0, activation_fn='tanh', is_training=train, norm_fn='None', use_bias=False, scope='conv_x')
            
            return x
        
##################################################################################
# Discriminator
##################################################################################

    def discriminator(self, x, reuse=False, train=True, dropout=False):
        norm='None'#'spectral#instance
        with tf.variable_scope("discriminator", reuse=reuse) :
        
            channel = self.ch
            for i in range(0, self.disn):
                x = conv(x, channel, kernel=4, stride=2, pad=1, activation_fn='leaky', 
                         is_training=train, norm_fn=norm, scope='conv_' + str(i))
                channel = channel * 2 if channel<512 else 512

            logit =  conv(x, 1, 3, stride=1, pad=1, activation_fn='None', 
                          is_training=train, norm_fn='None', use_bias=False, scope='D_logit')

            x = conv(x, self.label_size, kernel=int(x.shape[1]), stride=1, pad=0, activation_fn='None', 
                     is_training=train, norm_fn='None', use_bias=False, scope='D_label')
            
            x = tf.reshape(x, shape=[-1, self.label_size])

            return logit, x
        
##################################################################################
# Model
##################################################################################

    def gradient_panalty(self, real, fake, logit_real, logit_fake, scope="discriminator"):

        if self.gan_type == 'wgan-div':
            grad_real = tf.gradients(logit_real, real)[0]
            grad_fake = tf.gradients(logit_fake, fake)[0]
            grad_real_norm = tf.norm(tf.layers.flatten(grad_real), axis=1) # l2 norm
            grad_fake_norm = tf.norm(tf.layers.flatten(grad_fake), axis=1) # l2 norm
            GP = tf.reduce_mean(tf.pow(grad_real_norm, 3)+tf.pow(grad_fake_norm, 3))
            return GP
            
        if self.gan_type == 'dragan' :
            shape = tf.shape(real)
            eps = tf.random_uniform(shape=shape, minval=0., maxval=1.)
            x_mean, x_var = tf.nn.moments(real, axes=[0, 1, 2, 3])
            x_std = tf.sqrt(x_var)  # magnitude of noise decides the size of local region
            noise = 0.5 * x_std * eps  # delta in paper
            alpha = tf.random_uniform(shape=[shape[0], 1, 1, 1], minval=-1., maxval=1.)
            interpolated = tf.clip_by_value(real + alpha * noise, -1., 1.)  # x_hat should be in the space of X
        else :
            shape = tf.shape(real)
            alpha = tf.random_uniform(shape=[shape[0], 1, 1, 1], minval=0., maxval=1.)
            interpolated = alpha*real + (1. - alpha)*fake

        logit, _ = self.discriminator(interpolated, reuse=True)

        GP = 0

        grad = tf.gradients(logit, interpolated)[0] # gradient of D(interpolated)
        grad_norm = tf.norm(tf.layers.flatten(grad), axis=1) # l2 norm

        # WGAN - LP
        if self.gan_type == 'wgan-lp' :
            GP = tf.reduce_mean(tf.square(tf.maximum(0.0, grad_norm - 1.)))

        if self.gan_type == 'wgan-gp' or self.gan_type == 'dragan':
            GP = tf.reduce_mean(tf.square(grad_norm - 1.))

        return GP

    def optimizer_graph_generator(self, gen_loss, dis_loss, learning_rate_g, learning_rate_d, beta1):
        train_vars = tf.trainable_variables()
        gen_vars = [var for var in train_vars if var.name.startswith('generator')]
        dis_vars = [var for var in train_vars if var.name.startswith('discriminator')]
        # optimizer
        gen_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_g,beta1=beta1,beta2=0.999).minimize(gen_loss, var_list=gen_vars)
        dis_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_d,beta1=beta1,beta2=0.999).minimize(dis_loss, var_list=dis_vars)
        return gen_optimizer, dis_optimizer
    
    def build_model(self):
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
        self.input = tf.placeholder(tf.float32, shape=[None, 4, 4, 64], name='input')
        self.lr_g = tf.placeholder(tf.float32, name='lr_g')
        self.lr_d = tf.placeholder(tf.float32, name='lr_d')
        """ Dataset """
        self.Image_Data = ImageData(self.train_dataset_path, img_shape = self.img_shape, augment_flag = self.augment_flag, 
                                    data_type = self.train_dataset_type, img_type = self.train_dataset_img_type, label_size=self.label_size)

        trainA = tf.data.Dataset.from_tensor_slices((self.Image_Data.train_dataset, self.Image_Data.train_label))

        dataset_num = len(self.Image_Data.train_dataset)
        gpu_device = '/gpu:0'
        trainA = trainA.\
            apply(shuffle_and_repeat(dataset_num)).\
            apply(map_and_batch(self.Image_Data.image_processing, self.batch_size, num_parallel_batches=8, drop_remainder=True)).\
            apply(prefetch_to_device(gpu_device, self.batch_size))

        trainA_iterator = trainA.make_one_shot_iterator()

        self.real_imgs, self.label_o = trainA_iterator.get_next()
        
        """ generation """
        self.fake_imgs = self.generator(self.input, self.label_o)
                
        """ Discriminator for real """
        real_logits, real_label = self.discriminator(self.real_imgs)
        
        """ Discriminator for fake """
        fake_logits, fake_label = self.discriminator(self.fake_imgs, reuse=True)
        
        """ Define Loss """
        if self.gan_type.__contains__('wgan') or self.gan_type == 'dragan' :
            grad_pen = self.gradient_panalty(self.real_imgs, self.fake_imgs, real_logits, fake_logits)
        else :
            grad_pen = 0
        
        g_cls_loss = classification_loss2(logit=fake_label, label=self.label_o)
        d_cls_loss = classification_loss2(logit=real_label, label=self.label_o)
        
        dis_loss = discriminator_loss(self.gan_type, real_logits, fake_logits) + self.ld * grad_pen
        gen_loss = generator_loss(self.gan_type, fake_logits)
        
        D_loss = dis_loss + self.cls_weight * d_cls_loss
        G_loss = gen_loss + self.cls_weight * g_cls_loss
        """ Optimizer """
        D_loss += regularization_loss('discriminator')
        G_loss += regularization_loss('generator')
        self.gen_optimizer, self.dis_optimizer = self.optimizer_graph_generator(G_loss, D_loss, self.lr_g, self.lr_d, self.beta1)
        """ Summaries """
        self.g_summary = summary({G_loss:'G_loss',
                                  gen_loss: 'gen_loss',
                                  g_cls_loss:'g_cls_loss'})
        self.d_summary = summary({D_loss:'D_loss',
                                  dis_loss: 'dis_loss', 
                                  d_cls_loss:'d_cls_loss'})
        
    def train(self):
        
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()
    
        # summary writer
        self.writer = tf.summary.FileWriter(os.path.join(self.summaries_dir), self.sess.graph)
        
        # restore check-point if it exits
        could_load, checkpoint_counter = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_counter / self.iteration)
            start_batch_id = checkpoint_counter - start_epoch * self.iteration
            step = checkpoint_counter
            print(" [*] Load SUCCESS")
        else:
            if not os.path.exists(self.checkpoint_dir):
                os.makedirs(self.checkpoint_dir)
            start_epoch = 0
            start_batch_id = 0
            step = 1
            print(" [!] Load failed...")
        self.variables_count()
                
        lr = self.init_lr
        for epoch in range(start_epoch, self.epoch):
            lr = self.init_lr if epoch < self.decay_epoch else self.init_lr * (self.epoch - epoch) / (self.epoch - self.decay_epoch) # linear decay
    
            for idx in range(start_batch_id, self.iteration):
                input_z = np.random.uniform(-1, 1, size=(self.batch_size,4,4,64))
                train_feed_dict = {self.input : input_z, self.lr_g : lr, self.lr_d : lr}
                    
                _,d_summary_opt = self.sess.run([self.dis_optimizer, self.d_summary], feed_dict = train_feed_dict)
                self.writer.add_summary(d_summary_opt, step)
                if (step-1)%5==0:
                    _,g_summary_opt = self.sess.run([self.gen_optimizer, self.g_summary], feed_dict = train_feed_dict)
                    self.writer.add_summary(g_summary_opt, step)
                step += 1
                    
                if np.mod(idx + 1, self.save_freq) == 0:
                    samples_a, samples_t = self.sess.run([self.real_imgs,self.fake_imgs], feed_dict = train_feed_dict)
                    test_shape = (self.batch_size*self.img_size, self.img_size, self.img_ch)
                    samples_a=np.uint8(127.5*(np.reshape(samples_a,test_shape)+1.0))
                    samples_t=np.uint8(127.5*(np.reshape(samples_t,test_shape)+1.0))
                    sample = np.concatenate([samples_a,samples_t],axis=1)#,samples_p
                    sample = cv2.cvtColor(sample, cv2.COLOR_RGB2BGR) 
                    cv2.imwrite(os.path.join(self.sample_dir,str(step)+'.jpg'), sample)
        
                    print(datetime.now().strftime('%c'), ' epoch:', epoch, ' idx:', idx, '/', self.iteration, ' step:', step)
                    self.saver.save(self.sess, os.path.join(self.checkpoint_dir, self.model_name + '.model'), global_step=step)

            # After an epoch, start_batch_id is set to zero
            # non-zero value is only for the first epoch after loading pre-trained model
            start_batch_id = 0
    
            # save model for final step
            self.saver.save(self.sess, os.path.join(self.checkpoint_dir, self.model_name + '.model'), global_step=step)

    def test(self):
        
        self.label = tf.placeholder(tf.float32, [1, self.label_size], name='label')
        self.output = self.generator(self.input, self.label, reuse=True, train=False)
        
        # restore check-point if it exits
        self.saver = tf.train.Saver()
        could_load, checkpoint_counter = self.load(self.checkpoint_dir)
        if could_load:
            print(" [*] Load SUCCESS")
        else:
            print(" [!] Load failed...")

        class_names = ['D20', 'D10', 'D40', 'D00']
        gov_class = ['Czech', 'India', 'Japan']
        for gov in gov_class:
            file_list = os.listdir( self.test_path + gov + '/annotations/xmls')

            for file in file_list:

                im_name = file.split('.')[0] + '.jpg'

                full_impath =  self.test_path + gov + '/images/' + im_name

                save_path =  self.test_path + gov + '/images_x/'
                if not os.path.exists(save_path):
                    os.makedirs(save_path)

                infile_xml = open( self.test_path + gov + '/annotations/xmls/' + file, encoding='utf-8')
                tree = ElementTree.parse(infile_xml)
                root = tree.getroot()
                
                cnt = 0
                flag = 0
                for obj in root.iter('object'):
                    cls_name = obj.find('name').text
                    flag = 0
                    for n in class_names:
                        if cls_name == n:
                            flag+=1
                    if flag==0:
                        continue
                    xmlbox = obj.find('bndbox')
                    xmin = int(xmlbox.find('xmin').text)
                    xmax = int(xmlbox.find('xmax').text)
                    ymin = int(xmlbox.find('ymin').text)
                    ymax = int(xmlbox.find('ymax').text)

                    if xmin>xmax:
                        xmin = int(xmlbox.find('xmax').text)
                        xmax = int(xmlbox.find('xmin').text)

                    if ymin>ymax:
                        ymin = int(xmlbox.find('ymax').text)
                        ymax = int(xmlbox.find('ymin').text)

                    # open image
                    img = cv2.imread(full_impath)

                    label = np.reshape(self.Image_Data.get_class_one_hot(cls_name), [1, self.label_size]) 
                    input = np.random.uniform(-1, 1, size=(1,4,4,64))
                    img_o = self.sess.run([self.output], feed_dict={self.input: input, self.label: label})#
                    img_o = np.uint8(127.5*(np.reshape(img_o,(self.img_size, self.img_size, 3))+1.0))
                    img_o = cv2.cvtColor(img_o, cv2.COLOR_RGB2BGR) 
                    img_o = cv2.resize(img_o, (xmax-xmin, ymax-ymin))
                    width, height, channels = img.shape
                    # img_b = cv2.GaussianBlur(img,(5,5),0)
                    # img[ymin:ymax, xmin:xmax] = img_b[ymin:ymax, xmin:xmax]
                    obj = np.zeros_like(img)
                    obj[ymin:ymax, xmin:xmax] = img_o
                    mask = np.zeros_like(img)
                    mask[ymin:ymax, xmin:xmax] = 255
                    center = (xmin+(xmax-xmin)//2, ymin+(ymax-ymin)//2)
                    img = cv2.seamlessClone(obj, img, mask, center, cv2.MIXED_CLONE)
                if flag>0:
                    save_impath = save_path + im_name
                    cv2.imwrite(save_impath, img)
                    # cv2.imshow('img',mask)
                    # cv2.waitKey(0)
        

    def load(self, checkpoint_dir):
        print(" [*] Reading checkpoints...")
#        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            counter = int(next(re.finditer("(\d+)(?!.*\d)", ckpt_name)).group(0))
            print(" [*] Success to read {}".format(ckpt_name))
            return True, counter
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def set_value(self, matrix, x, y, val):
        w = int(matrix.get_shape()[0])
        h = int(matrix.get_shape()[1])
        mult_matrix = tf.sparse_tensor_to_dense(tf.SparseTensor(indices=[[x, y]], values=[-1.0], dense_shape=[w, h])) + 1.0
        diff_matrix = tf.sparse_tensor_to_dense(tf.SparseTensor(indices=[[x, y]], values=[val], dense_shape=[w, h]))
        matrix = tf.multiply(matrix, mult_matrix) 
        matrix = matrix + diff_matrix
        return matrix
    
    def variables_count(self):
        train_vars = tf.trainable_variables()
        gen_vars = [var for var in train_vars if var.name.startswith('generator')]
        dis_vars = [var for var in train_vars if var.name.startswith('discriminator')]
        print("Generator variables:", np.sum([np.prod(v.get_shape().as_list()) for v in gen_vars]))
        print("Discriminator variables:", np.sum([np.prod(v.get_shape().as_list()) for v in dis_vars]))
        print("Total variables:", np.sum([np.prod(v.get_shape().as_list()) for v in train_vars]))

In [ ]:
!mkdir -p IEEEbigdata2020/train

In [ ]:
!mv -v train/* IEEEbigdata2020/train

In [ ]:

# coding: UTF-8
import cv2
import os
from xml.etree import ElementTree


data_path = './IEEEbigdata2020/'
for gov in ['Czech', 'India', 'Japan']:
    file_list = os.listdir(data_path + 'train/' + gov + '/annotations/xmls')

    for file in file_list:

        im_name = file.split('.')[0] + '.jpg'

        full_impath = data_path + 'train/' + gov + '/images/' + im_name

        infile_xml = open(data_path + 'train/' + gov + '/annotations/xmls/' + file, encoding='utf-8')
        tree = ElementTree.parse(infile_xml)
        root = tree.getroot()
        
        cnt = 0
        for obj in root.iter('object'):
            
            cls_name = obj.find('name').text
            xmlbox = obj.find('bndbox')
            
            
            xmin = int(xmlbox.find('xmin').text)
            xmax = int(xmlbox.find('xmax').text)
            ymin = int(xmlbox.find('ymin').text)
            ymax = int(xmlbox.find('ymax').text)

            if xmin>xmax:
                xmin = int(xmlbox.find('xmax').text)
                xmax = int(xmlbox.find('xmin').text)

            if ymin>ymax:
                ymin = int(xmlbox.find('ymax').text)
                ymax = int(xmlbox.find('ymin').text)

            # open image
            img = cv2.imread(full_impath)

            
            # crop bounding box
            roi = img[ymin:ymax, xmin:xmax]

            if roi.sum() != 0:
                # resize image
                h = 128
                w = 128
                roiResized = cv2.resize(roi, (h, w))


                im_name = im_name.split('.')[0] + '_' + str(cnt) + '.png'
               # im_name = im_name.split('.')[0] + '.png'

                # save resized image
                save_dir = data_path + 'data/' + cls_name
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)
                cv2.imwrite(save_dir + '/' + im_name, roiResized)

                cnt = cnt + 1
                 
                im_name = file.split('.')[0] + '.jpg'
                